In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [4]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [5]:
def my_loss(real_data, gen_data):
    err = real_data - gen_data
    mae = tf.math.reduce_mean(tf.math.abs(err))
    y_pred = dnn_model(gen_data)

    loss = tf.math.add(tf.math.multiply(mae, 2), y_pred/30)
    return loss

In [6]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21)(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [7]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mae'])

In [8]:
gen = autoencoder.predict(X)

13/13 [==============================] - 0s 1ms/step


In [9]:
gen_en = encoder.predict(X)
pd_gen_en = pd.DataFrame(gen_en)
pd_gen_en.to_excel('ae_gen_en.xlsx')

13/13 [==============================] - 0s 1ms/step


In [10]:
dnn_model.predict(gen).mean()

13/13 [==============================] - 0s 1ms/step


39.75384

In [11]:
error

NameError: name 'error' is not defined

In [ ]:
gen_pop = dnn_model.predict(gen)
pd_gen_pop = pd.DataFrame(gen_pop)
pd_gen_pop.to_excel('ae_gen_pop.xlsx')

13/13 [==============================] - 0s 2ms/step


In [ ]:
inverse_gen = scaler.inverse_transform(gen)
pd_gen = pd.DataFrame(inverse_gen)
pd_gen.to_excel('ae_gen.xlsx')